In [2]:
# imports
import gymnasium as gym
from stable_baselines3 import PPO
from pong_Env import PingPongEnv

# Setup
env = PingPongEnv()
env.enemy_mode = "computer"

# Define Paramters
policy_kwargs = dict(net_arch=dict(pi=[256, 256], vf=[256, 256]))

print("--- STAGE 1 (Error 0.25) ---")
env.opponent_error_rate = 0.25

# Create Policy
model = PPO(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=policy_kwargs, 
    learning_rate=0.0003,
    batch_size=2048,
    n_steps=2048,
    ent_coef=0.01 
)

model.learn(total_timesteps=1000000)
model.save("pong_stage1")

# Defining Training stages
stages = [
    (0.15, "pong_stage2"),
    (0.05, "pong_stage3"),
    (0.0, "pong_champion_final")
]

for error_rate, save_name in stages:
    print(f"--- UPGRADING: Error Rate {error_rate} ---")
    
    # Load the previous stage to keep learning
    model = PPO.load("pong_stage1", env=env) # Load the one we just made
    
    env.opponent_error_rate = error_rate
    model.learn(total_timesteps=1500000)
    model.save(save_name)
    
    # Update the "base" model for the next loop
    model.save("pong_stage1") 

print("Training Complete")

--- STAGE 1 (Error 0.25) ---
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 202      |
|    ep_rew_mean     | -0.799   |
| time/              |          |
|    fps             | 5089     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 201          |
|    ep_rew_mean          | -0.8         |
| time/                   |              |
|    fps                  | 4537         |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0024031755 |
|    clip_fraction        | 0            |
|    clip_range      